## BigMart Sales Prediction

### Problem Statement

The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and find out the sales of each product at a particular store. 

Using this model, BigMart will try to understand the properties of products and stores which play a key role in increasing sales.

### Hypotheses

Let's categorize our hypothesis into following 2 categories: -

<b> Store Level Hypotheses:
1. City type: Stores located in urban or Tier 1 cities should have higher sales because of the higher income levels of people there.
2. Population Density: Stores located in densely populated areas should have higher sales because of more demand.
3. Store Capacity: Stores which are very big in size should have higher sales as they act like one-stop-shops and people would prefer getting everything from one place.
4. Competitors: Stores have similar establishments nearby should have less sales because of more competition.
5. Marketing: Stores which have a good marketing division should have higher sales as it will be able to attract customers through the right offers and advertising.
6. Location: Stores located within popular marketplaces should have higher sales because of better access to customers.
7. Customer Behavior: Stores keeping the right set of products to meet the local needs of customers will have higher sales.
8. Ambiance: Stores which are well-maintained and managed by polite and humble people are expected to have higher footfall and thus higher sales.

<b> Product Level Hypotheses:
1. Brand: Branded products should have higher sales because of higher trust in the customer.
2. Packaging: Products with good packaging can attract customers and sell more.
3. Utility: Daily use products should have a higher tendency to sell as compared to the specific use products.
4. Display Area: Products which are given bigger shelves in the store are likely to catch attention first and sell more.
5. Visibility in Store: The location of product in a store will impact sales. Ones which are right at entrance will catch the eye of customer first rather than the ones in back.
6. Advertising: Better advertising of products in the store should higher sales in most cases.
7. Promotional Offers: Products accompanied with attractive offers and discounts will sell more.

### Data Exploration

We will be performing some basic data exploration here and come up with some interences about the data.

Let's import libraries and load train and test dataset.

In [2]:
import pandas as pd
import numpy as np

#Read files:
train=pd.read_csv("Train.csv")
test=pd.read_csv("Test.csv")

Its generally a good idea to combine both train and test data sets into one, perform feature engineering and then divide them later again. This saves the trouble of performing the same steps twice on test and train. Lets combine them into a dataframe 'data' with a 'source' column specifying where each observation belongs.

In [4]:
train['source']='train'
test['source']='test'
data=pd.concat([train, test], ignore_index=True)
print (train.shape, test.shape, data.shape)

(8523, 13) (5681, 12) (14204, 13)


/home/priyesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


Thus we can see that data has same number of columns but rows equivalent to both test and train. One of the key challenges in any data set is missing values. Lets start by checking which columns contain missing values.

In [5]:
data.apply(lambda x: sum(x.isnull()))

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
source                          0
dtype: int64

Note that the Item_Outlet_Sales is the target variable and missing values are ones in the test set. So we need not worry about it. But we'll impute the missing values in Item_Weight and Outlet_Size in the data cleaning section.

Lets look at some basic statistics for numerical variables.

In [6]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


Some observations:

1. <b>Item_Visibility</b> has a min value of zero. This makes no practical sense because when a product is being sold in a store, the visibility cannot be 0.
2. <b>Outlet_Establishment_Years</b> vary from 1985 to 2009. The values might not be apt in this form. Rather, if we can convert them to how old the particular store is, it should have a better impact on sales.
3. The lower 'count' of Item_Weight and Item_Outlet_Sales confirms the findings from the missing value check.

Moving to nominal(categorical) variable, lets have a look at the number of unique values in each of them

In [7]:
data.apply(lambda x: len(x.unique()))

Item_Fat_Content                 5
Item_Identifier               1559
Item_MRP                      8052
Item_Outlet_Sales             3494
Item_Type                       16
Item_Visibility              13006
Item_Weight                    416
Outlet_Establishment_Year        9
Outlet_Identifier               10
Outlet_Location_Type             3
Outlet_Size                      4
Outlet_Type                      4
source                           2
dtype: int64

This tells us that there are <b>1559 products and 10 outlets/stores</b> (which was also mentioned in problem statement). Another thing that should catch attention is that <b>Item_Type has 16 unique values</b>. Let's explore further using the frequency of different categories in each nominal variable. I'll exclude the ID and source variables for obvious reasons.

In [10]:
#Filter categorical variables

categorical_columns= [x for x in data.dtypes.index if data.dtypes[x]=='object']

#Exclude ID cols and source:
categorical_columns= [x for x in categorical_columns if x not in ['Item_Identifier', 'Outlet_Identifier','source']]

#Print frequency of categories
for col in categorical_columns:
    print('\nFrequency of Categories for variable ',col)
    print(data[col].value_counts())


Frequency of Categories for variable  Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Frequency of Categories for variable  Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Frequency of Categories for variable  Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

Frequency of Categories for variable  Outlet_Size
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: 

The output gives us following observations:
1. <b>Item_Fat_Content:</b> Some of 'Low Fat' values mis-coded as 'low fat' and 'LF'.Also, some of 'Regular' are mentioned as 'regular'.
2. <b>Item_Type:</b> Not all categories have substantial numbers. It looks like combining them can give better results.
3. <b>Outlet_Type:</b> Supermarket Type2 and Type3 can be combined. But we should check if that's a good idea before doing it.

### Data Cleaning

This step typically involves imputing missing values and treating outliers. Though outlier removal is very important in regression techniques, advanced tree based algorithms are impervious to outliers. So I'll leave it to you to try it out. We'll focus on the imputation step here, which is a very important step.

#### <b>Imputing Missing Values

We found two variables with missing values - Item_Weight and Outlet_Size. Lets impute the former by the average weight of the particular item. This can be done as:

In [17]:
#Determine the average weight per item:
item_avg_weight= data.pivot_table(values='Item_Weight', index='Item_Identifier')

#Get a boolean variable specifying missing Item_Weight values
miss_bool=data['Item_Weight'].isnull()

#Impute data and check #missing values before and after imputation to confirm 
print("Original #missing: ", sum(miss_bool))
data.loc[miss_bool,'Item_Weight'] = data.loc[miss_bool,'Item_Identifier'].apply(lambda x: item_avg_weight.loc[x])
print("Final #missing: ", sum(data['Item_Weight'].isnull()))

Original #missing:  2439
Final #missing:  0


This confirms that the column has no missing values now. Lets impute Outlet_Size with the mode of the Outlet_Size for the particular type of outlet.